In [1]:
import DirectDmTargets as dddm

calculating effective efficiency for Xe detector done
calculating effective efficiency for Ar detector done
calculating effective efficiency for Ge detector done


In [2]:
import wimprates as wr
assert wr.__version__ !='0.2.2'
wr.__version__

'0.3.1'

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import numericalunits as nu

In [5]:
from tqdm import tqdm

In [6]:
from scipy.integrate import quad as scipy_int

In [7]:
import pandas as pd

In [8]:
import scipy

In [9]:
import emcee
emcee.__version__

'3.0.0'

In [10]:
import corner

In [11]:
import time

# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

# Emcee #
<https://emcee.readthedocs.io/en/stable/tutorials/quickstart/>

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [12]:
stats = dddm.MCMCStatModel("Xe")

stat_model::initialized for Xe detector. See print(stat_model) for default settings


In [13]:
stats.check_spectrum()

,counts,bin_centers,bin_left,bin_right
0,2.254830e+09,5.0,0.0,10.0
1,1.705936e+10,15.0,10.0,20.0
2,6.892848e+09,25.0,20.0,30.0
3,2.286659e+09,35.0,30.0,40.0
4,6.997966e+08,45.0,40.0,50.0
5,1.940946e+08,55.0,50.0,60.0
6,4.714355e+07,65.0,60.0,70.0
7,9.367342e+06,75.0,70.0,80.0
8,1.315625e+06,85.0,80.0,90.0
9,9.970640e+04,95.0,90.0,100.0


In [14]:
stats.config['prior']

{'log_mass': {'range': [0.1, 3],
  'prior_type': 'flat',
  'param': [0.1, 3],
  'dist': <function DirectDmTargets.statistics.get_priors.<locals>.<lambda>(x)>},
 'log_cross_section': {'range': [-46, -42],
  'prior_type': 'flat',
  'param': [-46, -42],
  'dist': <function DirectDmTargets.statistics.get_priors.<locals>.<lambda>(x)>},
 'density': {'range': [0.001, 0.9],
  'prior_type': 'gauss',
  'mean': 0.4,
  'std': 0.1,
  'param': (0.4, 0.1),
  'dist': <function DirectDmTargets.statistics.get_priors.<locals>.<lambda>(x)>},
 'v_0': {'range': [80, 380],
  'prior_type': 'gauss',
  'mean': 230,
  'std': 30,
  'param': (230, 30),
  'dist': <function DirectDmTargets.statistics.get_priors.<locals>.<lambda>(x)>},
 'v_esc': {'range': [379, 709],
  'prior_type': 'gauss',
  'mean': 544,
  'std': 33,
  'param': (544, 33),
  'dist': <function DirectDmTargets.statistics.get_priors.<locals>.<lambda>(x)>},
 'k': {'range': [0.5, 3.5],
  'prior_type': 'flat',
  'param': [0.5, 3.5],
  'dist': <function Di

In [15]:
def get_SHMpp_priors():
    """

    :return: dictionary of priors, type and values
    """
    priors = \
        {
        'log_mass':
            {'range': [0.1, 3], 'prior_type': 'flat'},
        'log_cross_section':
            {'range': [-46, -42], 'prior_type': 'flat'},
        'density':
            {'range': [0.001, 0.9], 'prior_type': 'gauss', 'mean': 0.55,
             'std': 0.17},
        'v_0':
            {'range': [80, 380], 'prior_type': 'gauss', 'mean': 230,
             'std': 3},
        'v_esc':
            {'range': [379, 709], 'prior_type': 'gauss', 'mean': 528,
             'std': 24.5},
        'k':
            {'range': [0.5, 3.5], 'prior_type': 'flat'}
        }
    for key in priors.keys():
        param = priors[key]
        if param['prior_type'] == 'flat':
            param['param'] = param['range']
            param['dist'] = lambda x: dddm.flat_prior(x)
        elif param['prior_type'] == 'gauss':
            param['param'] = param['mean'], param['std']
            param['dist'] = lambda x: dddm.gaus_prior(x)
    return priors

In [16]:
stats.config['prior'] = get_SHMpp_priors()

In [17]:
stats.config['prior']

{'log_mass': {'range': [0.1, 3],
  'prior_type': 'flat',
  'param': [0.1, 3],
  'dist': <function __main__.get_SHMpp_priors.<locals>.<lambda>(x)>},
 'log_cross_section': {'range': [-46, -42],
  'prior_type': 'flat',
  'param': [-46, -42],
  'dist': <function __main__.get_SHMpp_priors.<locals>.<lambda>(x)>},
 'density': {'range': [0.001, 0.9],
  'prior_type': 'gauss',
  'mean': 0.55,
  'std': 0.17,
  'param': (0.55, 0.17),
  'dist': <function __main__.get_SHMpp_priors.<locals>.<lambda>(x)>},
 'v_0': {'range': [80, 380],
  'prior_type': 'gauss',
  'mean': 230,
  'std': 3,
  'param': (230, 3),
  'dist': <function __main__.get_SHMpp_priors.<locals>.<lambda>(x)>},
 'v_esc': {'range': [379, 709],
  'prior_type': 'gauss',
  'mean': 528,
  'std': 24.5,
  'param': (528, 24.5),
  'dist': <function __main__.get_SHMpp_priors.<locals>.<lambda>(x)>},
 'k': {'range': [0.5, 3.5],
  'prior_type': 'flat',
  'param': [0.5, 3.5],
  'dist': <function __main__.get_SHMpp_priors.<locals>.<lambda>(x)>}}

In [18]:
for key in ['v_0', 'v_esc', 'rho_0']:
    if key == 'rho_0':
        key_prior = 'density'
    else:
        key_prior = key
    stats.config[key] = stats.config['prior'][key_prior]['mean']

In [19]:
stats.nwalkers = 250
stats.nsteps = 250
stats.log

{'sampler': False, 'did_run': False, 'pos': False}

In [ ]:
start = time.time()
stats.run_emcee()
end = time.time()

C:\Users\Joran\programming\Anaconda3\envs\DD_DM\lib\site-packages\emcee\ensemble.py:258: RuntimeWarning: Initial state is not linearly independent and it will not allow a full exploration of parameter space
  category=RuntimeWarning,
  0%|                                                                   | 0/250 [00:00<?, ?it/s]C:\Users\Joran\Google Drive\PhD-master\ubuntu-storage\windows-anaconda\DD_DM_targets\DirectDmTargets\statistics.py:150: RuntimeWarning: invalid value encountered in log10
  x = np.log10(x)
 56%|█████████████████████████████▍                       | 139/250 [1:07:32<1:07:05, 36.27s/it]

In [ ]:
end-start

In [ ]:
stats.save_results()

In [ ]:
assert stats.log['did_run']

In [ ]:
stats.show_walkers()

In [ ]:
stats.show_corner()

## Full dimensionality ##

In [ ]:
stats_full = dddm.MCMCStatModel("Xe")

In [ ]:
stats_full.config['prior'] = get_SHMpp_priors()
stats_full.config['v_0'] = stata_full.config['prior']['v_0']['mean']
stats_full.config['v_esc'] = stata_full.config['prior']['v_esc']['mean']
stats_full.config['rho_0'] = stata_full.config['prior']['density']['mean']

stats_full.config['prior']

In [ ]:
stats_full.nwalkers = stats.nwalkers
stats_full.nsteps = stats.nsteps * 2
stats_full.log

In [ ]:
stats_full.log

In [ ]:
# for key in stats_full.log.keys():
#     stats_full.log[key] = False


In [ ]:
stats_full.fit_parameters = stats_full.known_parameters

In [ ]:
start = time.time()
stats_full.run_emcee()
end = time.time()

In [ ]:
end-start

In [ ]:
stats_full.save_results(11)

In [ ]:
assert stats_full.log['did_run']

In [ ]:
stats_full.show_walkers()

In [ ]:
stats_full.show_corner()